<a href="https://colab.research.google.com/github/viktorcikojevic/dipolar-droplets/blob/main/derivation_of_variational_energy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here I will derive the variational energy for single-component dipolar system. 


Let's first import the dependencies

In [2]:
from sympy import *
from sympy.physics.quantum.constants import hbar

# Derivation of alpha and beta of the MF+LHY theory.

In [3]:
m, a_dd, a_s, rho, r_0 = symbols('m a_dd a_s rho r_0', real=True)
frac = Rational


# Mean-field contribution
g = 4 * pi * a_s * hbar**2 / m
eps_mf = g * rho**2 / 2
# LHY contribution
a_dd = r_0 / 3
eps_dd = a_dd / a_s
g_qf = 32 * g * a_s**frac('3/2') * (1 + 3*eps_dd**2 / 2) / (3*sqrt(pi)) 
eps_lhy = g_qf * rho**(frac('5/2')) * 2 / 5

# Total MF+LHY energy density
eps_mflhy = eps_mf + eps_lhy
eps_mflhy

256*hbar**2*sqrt(pi)*a_s**(5/2)*rho**(5/2)*(1 + r_0**2/(6*a_s**2))/(15*m) + 2*hbar**2*pi*a_s*rho**2/m

In [4]:
# Let's find the alpha and beta in this way: this is the target shape of mflhy functional. 
# Please check the cell below to conclude, since 
alpha, beta = symbols('alpha beta', real=True)
eps_target = alpha * rho + beta*rho**frac('5/2')
eps_target

alpha*rho + beta*rho**(5/2)

In [5]:
alpha = eps_mf / rho
beta = eps_lhy / rho**frac('5/2')
alpha

2*hbar**2*pi*a_s*rho/m

In [6]:
beta

256*hbar**2*sqrt(pi)*a_s**(5/2)*(1 + r_0**2/(6*a_s**2))/(15*m)

In [7]:
# Now let's find it in reduced units:
# rho is given as rho * r_0**3
eps_mflhy = eps_mflhy.subs(rho, rho/r_0**3)
eps_mflhy

256*hbar**2*sqrt(pi)*a_s**(5/2)*(rho/r_0**3)**(5/2)*(1 + r_0**2/(6*a_s**2))/(15*m) + 2*hbar**2*pi*a_s*rho**2/(m*r_0**6)

In [8]:
# Energy is given in units \hbar^2 / (m * r_0^2)
eps_mflhy = (eps_mflhy / (hbar**2 / (m * r_0**2) / r_0**3)).expand()
eps_mflhy = eps_mflhy.subs(a_s, a_s * r_0)
eps_mflhy

2*pi*a_s*rho**2 + 128*sqrt(pi)*r_0**7*sqrt(a_s*r_0)*(rho/r_0**3)**(5/2)/45 + 256*sqrt(pi)*r_0**5*(a_s*r_0)**(5/2)*(rho/r_0**3)**(5/2)/15

In [9]:
eps_mflhy

2*pi*a_s*rho**2 + 128*sqrt(pi)*r_0**7*sqrt(a_s*r_0)*(rho/r_0**3)**(5/2)/45 + 256*sqrt(pi)*r_0**5*(a_s*r_0)**(5/2)*(rho/r_0**3)**(5/2)/15

In [10]:
alpha = 2 * pi * a_s
alpha

2*pi*a_s

In [11]:
eps_mflhy

2*pi*a_s*rho**2 + 128*sqrt(pi)*r_0**7*sqrt(a_s*r_0)*(rho/r_0**3)**(5/2)/45 + 256*sqrt(pi)*r_0**5*(a_s*r_0)**(5/2)*(rho/r_0**3)**(5/2)/15

In [12]:
beta = (simplify(((eps_mflhy - alpha * rho**2) / rho**(frac('1/2'))/rho**2).expand()).subs(rho, 1)).expand().subs(r_0, 1)
beta

256*sqrt(pi)*a_s**(5/2)/15 + 128*sqrt(pi)*sqrt(a_s)/45

In [13]:
simplify(beta)

128*sqrt(pi)*sqrt(a_s)*(6*a_s**2 + 1)/45

In [14]:
print(beta)

256*sqrt(pi)*a_s**(5/2)/15 + 128*sqrt(pi)*sqrt(a_s)/45


In [15]:
# 256*np.sqrt(np.pi)*a_s**(5/2)/15 + 128*np.sqrt(np.pi)*np.sqrt(a_s)/45


# Derivation of MF+LHY energy per particle

In [16]:
N, s, z = symbols('N s z', real=True)
sigma_s, sigma_z = symbols('sigma_s, sigma_z', real=True)
alpha, beta, gamma = symbols('alpha, beta, gamma', real=True)

psi = sqrt(N) / (pi**(3/4) * sigma_s * sqrt(sigma_z)) * exp(-s**2/(sigma_s)**2/2 - z**2/(sigma_z)**2/2)
psi

pi**(-0.75)*sqrt(N)*exp(-s**2/(2*sigma_s**2) - z**2/(2*sigma_z**2))/(sigma_s*sqrt(sigma_z))

Double check to see if we get the correct norm.

In [17]:
norm = integrate(psi**2 * 2*pi*s, (s, 0, oo), conds='none')
norm = integrate(norm, (z, -oo, oo), conds='none')
norm

N

In [18]:
density = psi**2

In [19]:
gamma_term = beta*pow(density,(gamma + 1))
lhy_energy = exp(expand_log(log(gamma_term), force=True)).expand()
lhy_energy

pi**(-1.5)*N*beta*exp(-1.5*gamma*log(pi))*exp(gamma*log(N))*exp(-2*gamma*log(sigma_s))*exp(-gamma*log(sigma_z))*exp(-s**2/sigma_s**2)*exp(-z**2/sigma_z**2)*exp(-gamma*s**2/sigma_s**2)*exp(-gamma*z**2/sigma_z**2)/(sigma_s**2*sigma_z)

In [20]:

energy_delta = alpha * density**2 + lhy_energy
# energy_delta = energy_delta.expand()
energy_delta

pi**(-3.0)*N**2*alpha*exp(-2*s**2/sigma_s**2 - 2*z**2/sigma_z**2)/(sigma_s**4*sigma_z**2) + pi**(-1.5)*N*beta*exp(-1.5*gamma*log(pi))*exp(gamma*log(N))*exp(-2*gamma*log(sigma_s))*exp(-gamma*log(sigma_z))*exp(-s**2/sigma_s**2)*exp(-z**2/sigma_z**2)*exp(-gamma*s**2/sigma_s**2)*exp(-gamma*z**2/sigma_z**2)/(sigma_s**2*sigma_z)

In [21]:
energy_delta = expand(simplify(energy_delta / N))
energy_delta

pi**(-3.0)*N*alpha*exp(-2*s**2/sigma_s**2)*exp(-2*z**2/sigma_z**2)/(sigma_s**4*sigma_z**2) + pi**(-1.5)*pi**(-1.5*gamma)*beta*exp(gamma*log(N))*exp(-2*gamma*log(sigma_s))*exp(-gamma*log(sigma_z))*exp(-s**2/sigma_s**2)*exp(-z**2/sigma_z**2)*exp(-gamma*s**2/sigma_s**2)*exp(-gamma*z**2/sigma_z**2)/(sigma_s**2*sigma_z)

In [22]:
en_mflhy = integrate(energy_delta, (z, -oo, oo), conds='none')
en_mflhy = integrate(en_mflhy *  2*pi*s, (s, 0, oo), conds='none')
en_mflhy

sqrt(2)*pi**(-1.5)*N*alpha/(4*sigma_s**2*sigma_z) + pi**(-1.5*gamma)*beta*exp(gamma*log(N) - 2*gamma*log(sigma_s) - gamma*log(sigma_z))/(gamma + 1)**(3/2)

Great! Let's print it so we can plug it into our $n_c$ solver

In [23]:
print(en_mflhy)

sqrt(2)*pi**(-1.5)*N*alpha/(4*sigma_s**2*sigma_z) + pi**(-1.5*gamma)*beta*exp(gamma*log(N) - 2*gamma*log(sigma_s) - gamma*log(sigma_z))/(gamma + 1)**(3/2)


In [24]:
# en_mflhy = np.sqrt(2)*np.pi**(-1.5)*N*alpha/(4*sr**2*sz) + np.pi**(-1.5*gamma)*beta*np.exp(gamma*np.log(N) - 2*gamma*np.log(sr) - gamma*np.log(sz))/(gamma + 1)**(3/2)



# Dipolar part

OK, now that we've got the easy part done, let's estimate the dipolar interaction energy